In [ ]:
# hide
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# default_exp midi

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import Optional, Tuple, Collection
import re

from music21 import converter, instrument, note, chord, stream, volume, duration, midi
from music21.midi import MidiException
import pandas as pd

<IPython.core.display.Javascript object>

In [ ]:
from test import test_eq

<IPython.core.display.Javascript object>

# Representing MIDI notes as triplets

Although MIDI notes have a lot of information, we want to distill them into a triplet -- namely pitch, duration and velocity:

In [ ]:
# export

Pitch, Duration, Velocity = str, float, int
Triplet = Tuple[Pitch, Duration, Velocity]

<IPython.core.display.Javascript object>

First we'll create some functions to turn single notes, chords and rests into these Triplets.

In [ ]:
# export


def parse_single_note(note: note.Note) -> Triplet:
    return (str(note.pitch), float(note.duration.quarterLength), note.volume.velocity)


def parse_chord(chord: chord.Chord) -> Triplet:
    return (
        ".".join(str(n) for n in chord.normalOrder),
        float(chord.duration.quarterLength),
        chord.volume.velocity,
    )


def parse_rest(rest: note.Rest) -> Triplet:
    return ("R", float(rest.duration.quarterLength), 0)

<IPython.core.display.Javascript object>

In [ ]:
# test
quarter = duration.Duration("quarter")


def loud(n: note.Note, velocity=120) -> note.Note:
    n.volume = volume.Volume(velocity=velocity)
    return n


test_eq(("C3", 1.0, 120), parse_single_note(loud(note.Note("C3", duration=quarter))))
test_eq(
    ("1.3.5", 1.0, 80),
    parse_chord(loud(chord.Chord([1, 3, 5], duration=quarter), velocity=80)),
)
test_eq(("R", 1.0, 0), parse_rest(note.Rest(duration=quarter)))

<IPython.core.display.Javascript object>

In [ ]:
# export


def parse_midi_file(file: str) -> (Optional[pd.DataFrame], int):
    """
    Attempts to parse a midi file into a Dataframe. Returns a Dataframe or None, together with the number of notes processed.
    """
    try:
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        notes = []
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if element.__class__ == note.Note:
                notes.append(parse_single_note(element))
            elif element.__class__ == chord.Chord:
                notes.append(parse_chord(element))
            elif element.__class__ == note.Rest:
                notes.append(parse_rest(element))

        df = pd.DataFrame.from_dict(
            {
                "pitches": [[[pitch for (pitch, dur, velocity) in notes]]],
                "durations": [[[dur for (pitch, dur, velocity) in notes]]],
                "velocities": [[[velocity for (pitch, dur, velocity) in notes]]],
            }
        )
        return (df, len(notes))
    except MidiException:
        return (None, 0)
    except IndexError:
        return (None, 0)


def row_to_triplets(df: pd.DataFrame, row_index: int) -> Collection[Triplet]:
    """
    Takes a DataFrame at a specific row and turns that into a triplet.
    """
    pitches, durations, velocities = df.values[row_index]
    pitches = pitches[0]
    durations = durations[0]
    velocities = velocities[0]
    return list(zip(pitches, durations, velocities))

<IPython.core.display.Javascript object>

In [ ]:
# test
df, notes = parse_midi_file("data/ff4-main.mid")

test_eq(812, notes)

triplets = row_to_triplets(df, 0)

test_eq([("A5", 0.5, 110), ("R", 192.0, 0), ("E5", 0.5, 110)], triplets[0:3])

<IPython.core.display.Javascript object>

# Turning triplets back into MIDI files

Another interesting thing is to be able to go from triplets to MIDI files, display their scores and even an audible IPython widget to play the audio.

But first of all, some utilities to turn these triplets into `music21` elements:

In [ ]:
# export


def triplet_to_note(triplet: Triplet):
    pitch, dur, vel = triplet
    dur = duration.Duration(dur)
    vol = volume.Volume(velocity=vel)
    if re.match(r"^[0-9]", pitch):
        c = chord.Chord([int(x) for x in pitch.split(".")], duration=dur)
        c.volume = vol
        return c
    elif pitch == "R":
        return note.Rest(duration=dur)
    else:
        n = note.Note(pitch, duration=dur)
        n.volume = vol
        return n


def triplets_to_stream(triplets: Collection[Triplet]) -> stream.Stream:
    s = stream.Stream()
    _ = [s.append(triplet_to_note(triplet)) for triplet in triplets]
    return s


def write_midi(stream, name):
    mf = midi.translate.streamToMidiFile(stream)
    mf.open(f"{name}.mid", "wb")
    mf.write()
    mf.close()

<IPython.core.display.Javascript object>

Now let's try to visualize and listen to these streams.

If you're running Mac or Linux, uncomment the right `brew` or `apt` call:

In [ ]:
# skip

#!brew install fluidsynth
#!apt install fluidsynth
#!wget ftp://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf3
#!pip install midi2audio

--2019-12-08 01:55:32--  ftp://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General/MuseScore_General.sf3
           => ‘MuseScore_General.sf3’
Resolving ftp.osuosl.org (ftp.osuosl.org)... 140.211.166.134, 64.50.236.52, 64.50.233.100
Connecting to ftp.osuosl.org (ftp.osuosl.org)|140.211.166.134|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/musescore/soundfont/MuseScore_General ... done.
==> SIZE MuseScore_General.sf3 ... 39893918
==> PASV ... done.    ==> RETR MuseScore_General.sf3 ... done.
Length: 39893918 (38M) (unauthoritative)

MuseScore_General.s 100%[===================>]  38.04M  4.47MB/s    in 12s     

2019-12-08 01:55:48 (3.23 MB/s) - ‘MuseScore_General.sf3’ saved [39893918]



<IPython.core.display.Javascript object>

In [ ]:
# skip

%load_ext music21.ipython21.ipExtension

import json, random

from IPython.core.display import display, HTML, Javascript
from IPython.display import Audio

from midi2audio import FluidSynth


def show_score(score):
    xml = open(score.write("musicxml")).read()
    show_xml(xml)


def show_xml(xml):
    DIV_ID = "OSMD-div-" + str(random.randint(0, 1000000))
    # print("DIV_ID", DIV_ID)
    msg = "loading OpenSheetMusicDisplay"
    msg = ""
    display(
        HTML('<div style="background: white" id="' + DIV_ID + '">{}</div>'.format(msg))
    )

    # print('xml length:', len(xml))

    script = """
    console.log("loadOSMD()");
    function loadOSMD() { 
        return new Promise(function(resolve, reject){

            if (window.opensheetmusicdisplay) {
                console.log("already loaded")
                return resolve(window.opensheetmusicdisplay)
            }
            console.log("loading osmd for the first time")
            // OSMD script has a 'define' call which conflicts with requirejs
            var _define = window.define // save the define object 
            window.define = undefined // now the loaded script will ignore requirejs
            var s = document.createElement( 'script' );
            s.setAttribute( 'src', "https://cdn.jsdelivr.net/npm/opensheetmusicdisplay@0.3.1/build/opensheetmusicdisplay.min.js" );
            //s.setAttribute( 'src', "/custom/opensheetmusicdisplay.js" );
            s.onload=function(){
                window.define = _define
                console.log("loaded OSMD for the first time",opensheetmusicdisplay)
                resolve(opensheetmusicdisplay);
            };
            document.body.appendChild( s ); // browser will try to load the new script tag
        }) 
    }
    loadOSMD().then((OSMD)=>{
        console.log("loaded OSMD",OSMD)
        var div_id = "{{DIV_ID}}";
            console.log(div_id)
        window.openSheetMusicDisplay = new OSMD.OpenSheetMusicDisplay(div_id);
        openSheetMusicDisplay
            .load({{data}})
            .then(
              function() {
                console.log("rendering data")
                openSheetMusicDisplay.render();
              }
            );
    })
    """.replace(
        "{{DIV_ID}}", DIV_ID
    ).replace(
        "{{data}}", json.dumps(xml)
    )
    display(Javascript(script))
    return DIV_ID


def view_song(triplets, name="song"):
    stream = triplets_to_stream(triplets)
    write_midi(stream, f"{name}")

    FluidSynth("MuseScore_General.sf3").midi_to_audio(f"{name}.mid", f"{name}.wav")
    show_score(stream)
    return Audio(f"{name}.wav")

<IPython.core.display.Javascript object>

In [ ]:
# skip
triplets = row_to_triplets(df, 0)
view_song(triplets)